<a href="https://colab.research.google.com/github/ccwu0918/MathProgramming/blob/main/Appendix/Appendix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 附錄_程式設計與數學之間的橋樑
在此要介紹的是於學習本書介紹的數學之際，執行必要程式的流程。

In [ ]:
#Colaboratory環境的設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Appendix

In [ ]:
#函式庫的設定
!pip install -q -r ./requirements.txt

## A-1 利用公式了解常態分佈

### 定義機率密度函數

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

# 定義常態分佈
def normal_distribution(x,mu,sigma):
    y = 1/np.sqrt(2*np.pi*sigma**2)*np.exp(-(x-mu)**2/(2*sigma**2))
    return y

### 繪製機率密度函數

In [ ]:
# 設定常態分佈的參數
mu = 116.6
sigma = 4.8

# 設定繪圖參數
x_min = 80
x_max = 150
x_num = 100

# 計算常態分佈
x = np.linspace(x_min, x_max, x_num)
y = normal_distribution(x,mu,sigma)

# 繪製常態分佈
plt.plot(x, y ,color="k")
plt.show()
%matplotlib inline

### 根據機率密度函數的積分導出面積

In [ ]:
# 設定積分範圍
x_min = 115
x_max = 117
x_num = 100

# 計算積分範圍之內的常態分佈值
x = np.linspace(x_min, x_max, x_num)
y = normal_distribution(x,mu,sigma)

# 積分的計算
dx = (x_max-x_min)/(x_num-1)
prob = 0
for i in range(x_num):
    y = normal_distribution(x[i],mu,sigma)
    prob += y*dx
print("機率:",prob)

## A-2微分方程式差分法造成的誤差與泰勒展開式

In [ ]:
from sympy import*
import numpy as np
from matplotlib import pyplot as plt

# 設定參數
n = 1     # 次數
x0 = 0    # 初始值

# 定義符號
x = Symbol('x')

# 定義函數
f = 2 + x + sin(x) + exp(x)/10

# 導出泰勒展開式
taylor = series(f, x=x, x0=x0, n=n+1).removeO()
taylor_y = lambdify(x, taylor, 'numpy')
print("泰勒展開式")
print(taylor)

# 繪製圖表
x_theory = np.arange(0.0, 10.0, 0.1)
y_theory = 2+x_theory+np.sin(x_theory)+np.exp(x_theory)/10
plt.plot(x_theory, y_theory, lw=3, c="k")
plt.plot(x_theory, taylor_y(x_theory),c="b")
plt.xlim([0,10])

## 11-3 非線性最佳化的機械學習／深度學習的迴歸／分類

### 定義（非線形最佳化）函數

In [ ]:
def function(x):
    y = x**2
    return y

def differential(x,dx):
    dy = (function(x+dx)-function(x))/dx
    return dy

### 執行（非線形最佳化）梯度下降法

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

# 產生函數
x_list = np.arange(-10, 11)
y_list = function(x_list)
num = len(x_list)

# 設定參數
dx = 0.1     # 刻度的寬度（學習率）
iter = 200   # 重覆次數

# 設定初始值
x = -10

# 迴圈處理
list_plot = []
fig = plt.figure()
for t in range(iter):
    # 導出導函數
    dy = differential(x,dx)
    # 更新x,y
    x = x - np.sign(dy)*dx
    y = function(x)
    # 繪製圖表
    img = plt.plot(x,y,marker='.', color="red",markersize=20)
    img += plt.plot(x_list,y_list,color="black")
    list_plot.append(img)

# 繪製圖表（動畫）
plt.grid()
anim = animation.ArtistAnimation(fig, list_plot, interval=200, repeat_delay=100)
rc('animation', html='jshtml')
plt.close()
anim

### （非線形最適化）処理部のみ実行

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 設定參數
delta = 0.01     # 刻度的寬度（學習率）
iter = 200       # 重覆次數

# 設定初始值
x = -10

# 迴圈處理
list_plot = []
series_y = []
fig = plt.figure()
for t in range(iter):
    # 導出導函數
    dy = differential(x,dx)
    # 更新x,y
    x = x - delta*dy
    y = function(x)
    series_y.append(y)

# 繪製圖表
plt.plot(series_y,c="k")

### 定義（迴歸分析）函數

In [ ]:
def function(X,y,alpha,beta):
    cost = (1/(2*m))*np.sum((beta+alpha*X-y)**2)
    return cost

def differential_alpha(X,y,alpha,beta,delta):
    d_cost = (function(X,y,alpha+delta,beta)-function(X,y,alpha,beta))/delta
    return d_cost

def differential_beta(X,y,alpha,beta,delta):
    d_cost = (function(X,y,alpha,beta+delta)-function(X,y,alpha,beta))/delta
    return d_cost

### 載入（迴歸分析）資料

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 載入資料
df_sample = pd.read_csv("sample_linear.csv")
sample = df_sample.values.T

# 設定變數
X = sample[0]
y = sample[1]

### 執行（迴歸分析）梯度下降法

In [ ]:
# 設定參數
delta = 0.001     # 刻度的寬度（學習率）
iter = 20000      # 重覆次數

# 設定初始值
alpha = 1
beta = 1

# 迴圈處理
cost = np.zeros(iter)
da = np.zeros(iter)
m = len(y)
for i in range(iter):

    # 導出導函數
    d_alpha = differential_alpha(X,y,alpha,beta,delta)
    d_beta = differential_beta(X,y,alpha,beta,delta)

    # 更新alpha, beta, cost
    alpha = alpha - delta*d_alpha
    beta = beta - delta*d_beta
    cost[i] = function(X,y,alpha,beta)
    da[i] = alpha

# 繪製圖表
plt.plot(da,c="k")

### 繪製（迴歸分析）最小平方法的結果

In [ ]:
plt.scatter(sample[0],sample[1],c="k")
plt.plot(X,beta+alpha*X,color="red")
plt.show()